# Create Zarr of 9 years of six hourly CSSP China data [1851-1859]

In [7]:
import iris
import os
import sys
import logging
import xarray as xr
import numpy as np

import crd_utils as crd
import umdates_utils as um

from datetime import datetime, timedelta

## Create a list of all the files we want to process

In [8]:
# six hourly data filenames
filepath = '/data/cssp-china/mini-dataset-24-01-19/20CR/6hrly'
files = sorted(os.listdir(filepath))
print(len(files))

5850


In [9]:
# generate all possible filenames for the time period
runid = 'apepd'
startd = datetime(1851, 1, 1)   # 00Z on Jan 01 1851
endd = datetime(1860, 1, 1)     # 00Z on Jan 01 1860
freq = 'pb'

decade_filenames = um.UMFileList(runid, startd, endd, freq)
print(len(decade_filenames))

3288


In [10]:
# find the overlap of actual filenames with all possible filenames in that decade
filenames = list(set(files).intersection(set(decade_filenames)))
filenames.sort()
print(len(filenames))

329


In [11]:
filepaths = [os.path.join(filepath, filename) for filename in filenames]
print(len(filepaths))

329


## Set all the Cube, Dataset and Zarr variables we need to process the cubes

In [12]:
%%time
cubelist = iris.load(filepaths[0:1])
cubelist[0]

CPU times: user 5.47 s, sys: 513 ms, total: 5.98 s
Wall time: 7.43 s


<iris 'Cube' of m01s05i217 / (unknown) (time: 16; grid_latitude: 219; grid_longitude: 286)>

In [13]:
print(cubelist)

0: m01s05i217 / (unknown)              (time: 16; grid_latitude: 219; grid_longitude: 286)
1: m01s05i217 / (unknown)              (time: 16; grid_latitude: 219; grid_longitude: 286)
2: air_temperature / (K)               (time: 17; pressure: 14; grid_latitude: 219; grid_longitude: 286)
3: geopotential_height / (m)           (time: 17; pressure: 14; grid_latitude: 219; grid_longitude: 286)
4: lagrangian_tendency_of_air_pressure / (Pa s-1) (time: 17; pressure: 14; grid_latitude: 218; grid_longitude: 286)
5: relative_humidity / (%)             (time: 16; grid_latitude: 219; grid_longitude: 286)
6: specific_humidity / (1)             (time: 16; grid_latitude: 219; grid_longitude: 286)
7: surface_temperature / (K)           (time: 17; grid_latitude: 219; grid_longitude: 286)
8: x_wind / (m s-1)                    (time: 17; grid_latitude: 218; grid_longitude: 286)
9: x_wind / (m s-1)                    (time: 17; pressure: 14; grid_latitude: 218; grid_longitude: 286)
10: y_wind / (m s-1)   

In [14]:
CUBENAMES = [cube.name() for cube in cubelist]
UNIQUE_COORDS = crd.unique_coords_list(cubelist)
COORD_NAME_MAPPING = crd.get_new_coord_names(UNIQUE_COORDS)

In [15]:
print([coord.name() for coord in UNIQUE_COORDS])

['time', 'grid_latitude', 'grid_longitude', 'forecast_reference_time', 'forecast_period', 'time', 'pressure', 'forecast_period', 'grid_latitude', 'grid_longitude', 'height', 'height']


In [16]:
COORD_NAME_MAPPING[1]

('time_1',
 'forecast_period_1',
 'grid_latitude_1',
 'grid_longitude_1',
 'height_1')

In [17]:
ZARR = '/data/cssp-china/zarr_6hourly_1851-1859'
CHUNKS = {'time': 200, 'grid_latitude': 219, 'grid_longitude': 286, 'grid_latitude_1': 218, 'grid_longitude_1': 286}

## Initialise logging

In [14]:
LOGFILE = '/data/cssp-china/zarr_append_hourly.log'
logging.basicConfig(filename=LOGFILE,
                    level=logging.DEBUG,
                    format='%(asctime)s %(message)s',
                    datefmt='%d/%m/%Y %H:%M:%S')

In [15]:
if not os.path.isfile(LOGFILE):
    os.mknod(LOGFILE)

if os.stat(LOGFILE).st_size == 0:
    logging.info('Initiate log')
    print(f'Log initiated at {LOGFILE}')

Log initiated at /data/cssp-china/zarr_append_hourly.log


## Create a Zarr with 3 cubes, which definitely amounts to 577 time steps (~25 days)

In [18]:
%%time
cubelist0 = iris.load(filepaths[0:3])
cubelist0[1]

CPU times: user 29.7 s, sys: 4.18 s, total: 33.9 s
Wall time: 40 s


<iris 'Cube' of m01s05i217 / (unknown) (time: 96; grid_latitude: 219; grid_longitude: 286)>

In [19]:
%%time
crd.rename_cubes(cubelist0, CUBENAMES, COORD_NAME_MAPPING, dryrun=False)

m01s05i217 -> m01s05i217_max
  x time
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x forecast_period
m01s05i217 -> m01s05i217_mean
  x time
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x forecast_period
air_temperature
  x time
  x pressure
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x forecast_period
geopotential_height
  x time
  x pressure
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x forecast_period
lagrangian_tendency_of_air_pressure
  x time
  x pressure
    grid_latitude_1
    grid_longitude_1
  x forecast_reference_time
  x forecast_period
relative_humidity
  x time
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x height
  x forecast_period
specific_humidity
  x time
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x height
  x forecast_period
surface_temperature
  x time
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x forecast_per

# BREAK: Two time axes that aren't picked up by the coord renaming

----------

In [ ]:
%%time
dalist0 = crd.cubelist_to_dalist(cubelist0)
ds0 = xr.merge(dalist0)

In [ ]:
ds0

In [ ]:
%%time
logging.info(f'Creating {ZARR}')
crd.ds_to_zarr(ds0, ZARR, chunks=CHUNKS)

In [ ]:
dsz0 = xr.open_zarr(ZARR)
dsz0

In [ ]:
# Size of Dataset in GB
dsz0.nbytes / 1e9

In [ ]:
logging.info(f'  Successfull creation of {ZARR}')
logging.info(f'  Processed filenames 0:{filenames[0]} - 3:{filenames[3]}')
logging.info(f'  Chunking {CHUNKS}')

## Now loop through the remaining cubes and append to the Zarr we created

In [ ]:
def logprint(message):
    logging.info(message)
    print(message)

In [ ]:
%%time
# Loop through in bunches of 2 cubes, so as to not use to much memory at one time
start = 3
step = 2
# stop = len(filepaths)
stop = start+(2*step)

logprint(f'Appending data to {ZARR}')

for i in range(start, stop, step):
    fnames = filenames[i:i+step]
    logprint(f'  Processing filenames {i}:{fnames[0]} - {min([i+step, stop])}:{fnames[-1]}')
    try:
        cubelist = iris.load(filepaths[i:i+step])
        logprint(f'  | Loaded files successfully')
        
        crd.rename_cubes(cubelist, CUBENAMES, COORD_NAME_MAPPING, dryrun=False, verbose=False)
        logprint(f'  | Renamed files successfully')
        
        dalist = crd.cubelist_to_dalist(cubelist)
        ds = xr.merge(dalist)
        logprint(f'  | Created dataset successfully')
        
        crd.ds_to_zarr(ds, ZARR, chunks=CHUNKS)
        logprint(f'  | Appended to Zarr {ZARR} successfully')
        
    except Exception as e:
        logprint(f'  X ERROR: {e}')
        raise e

logprint(f'  Appending data complete')

In [ ]:
dsz1 = xr.open_zarr(ZARR)
dsz1

In [ ]:
cubez = dsz1.surface_air_pressure.to_iris()
cubez

In [ ]:
deltas = cubez.coord('time').points[1:]-cubez.coord('time').points[0:-1]
deltas

In [ ]:
list(deltas).index(240)